In [4]:
import os
import json

chunks = []

folder = r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/data/storage/chunks_metadata.json"

with open(folder, "r", encoding="utf-8") as f:
    data = json.load(f)
    if isinstance(data, list):
        chunks.extend(data)
    elif isinstance(data, dict):
        chunks.append(data)
    else:
        print(f"Skipping {folder} as it is not a list or dictionary.")

print(f"✅ Loaded {len(chunks)} chunks from {folder}")

texts = [c["content"] for c in chunks]
metadata = [
    {
        "id": c["id"],
        "title": c["title"],
        "source": c["source"],
        "text": c["content"]
    }
    for c in chunks
]

✅ Loaded 100 chunks from /Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/data/storage/chunks_metadata.json


In [ ]:
import openai
import numpy as np
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

texts = [c["content"] for c in chunks]
embeddings = []

for text in tqdm(texts, desc="Embedding texts"):
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    emb = np.array(response.data[0].embedding, dtype="float32")
    emb /= np.linalg.norm(emb)
    embeddings.append(emb)

embeddings = np.vstack(embeddings)


np.save("/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/data/storage/embeddings.npy", embeddings)

with open("texts.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text.replace("\n", " ") + "\n")

Embedding texts: 100%|██████████| 100/100 [00:43<00:00,  2.28it/s]


In [21]:
import faiss
import os

embeddings = np.load("/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/data/storage/embeddings.npy")

dim = embeddings.shape[1]

index = faiss.IndexFlatIP(dim)
index.add(embeddings)

faiss.write_index(index, "/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/data/storage/index.faiss")

print(f"✅ FAISS index created and saved with {index.ntotal} vectors.")


✅ FAISS index created and saved with 100 vectors.


In [22]:
def get_embedding(text: str, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    embedding = response.data[0].embedding
    return np.array(embedding, dtype='float32')

In [ ]:
import json
metadata = [
    {
        "id": i,
        "title": c.get("title", ""),
        "source": c.get("source", ""),
        "content": c["content"]
    }
    for i, c in enumerate(chunks)
]


with open("storage/chunks_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

In [24]:
import json
import faiss
import openai
import numpy as np

index = faiss.read_index(r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/data/storage/index.faiss")

with open(r"/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/data/storage/chunks_metadata.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)

def search_index(query, k=5, min_score=0.4):
    query_vector = get_embedding(query).reshape(1, -1)
    query_vector /= np.linalg.norm(query_vector)

    distances, indices = index.search(query_vector, k)

    results = []
    top_pairs = list(zip(distances[0], indices[0]))

    for dist, idx in top_pairs:
        if idx < 0:
            continue
        if dist >= min_score:
            chunk_data = metadata[idx]
            results.append({
                "score": float(dist),
                "chunk": chunk_data["content"],
                "metadata": {
                    "id": chunk_data["id"],
                    "title": chunk_data.get("title", ""),
                    "source": chunk_data.get("source", "")
                }
            })

    # Fallback: if no result passed the min_score, return best match
    if not results and top_pairs:
        dist, idx = top_pairs[0]
        if idx >= 0:
            chunk_data = metadata[idx]
            results.append({
                "score": float(dist),
                "chunk": chunk_data["content"],
                "metadata": {
                    "id": chunk_data["id"],
                    "title": chunk_data.get("title", ""),
                    "source": chunk_data.get("source", "")
                }
            })

    return results

query = "ما هي العلامات التي تدل على استجابة دعاء الإنسان؟"
query = "كيف يمكن للإنسان أن يعرف أن دعاءه قد استجيب؟"

results = search_index(query)

print("Top relevant chunks:")
for i, res in enumerate(results, 1):
    print(f"\nResult {i} (score: {res['score']:.4f}):")
    print(res["chunk"])
    print(res["metadata"])


Top relevant chunks:

Result 1 (score: 0.4972):
ثم لا يتغير ولا يتبدل ولا يتأثر ولا ينفعل. ما هذا قلب محجوب وهذا قلب لا يشعر هذه لا يشعر بهذا الدعاء وأهميته بل لا يستطعن طعمه ولذته. لان لذة الدعاء تقربك من الله سبحانه وتعالى. ولذا ورد عندنا ان في بعض بعض من يدعو. بعض الروايات ورد عندنا أن من يدعو الله عز وجل هو يعرف في قرارة نفسه وفي خاتمة دعائه أن دعاؤه وصل أو لم يصل. كيف؟ إذا شعر في قلبه انفعالا. وفي بعض الروايات تتحدث عن قطرة دمع تنزل من عينيه. هذه إشارة على استجابة الدعاء وعلى وصول الدعاء لأن هناك انفعالا ولأن هناك تأثرا على أي حال.
{'id': 56, 'title': 'عاشوراء 2016 - 1438 هجري » كلمة السيد هاشم صفي الدين في الليلة السادسة من شهر محرم الحرام.', 'source': 'موقع السيد هاشم صفي الدين https://hashem-safieddine.net/gallery/preview.php?file_id=290'}

Result 2 (score: 0.4253):
حياتنا هوى. حياتنا شهوة. حياتنا نزوة حياتنا. رغبة حينئذ نحن محجوبون عن الله عز وجل. حينما يغلبنا الهوى. الدعاء لا ينفع. التوسل لا ينفع. الرجوع إلى الله بعد إذن لا ينفع. لا بد أن تقلع عن هذا الهوى حتى يستقيم الأمر. و

In [6]:
def reformulate_query(query):
    model="gpt-4o"
    system_prompt = (
        "أنت مساعد متخصص في إعادة صياغة الأسئلة بطريقة مهنية ضمن نظام استرجاع المعلومات (RAG)."
        "إذا كان السؤال مكتوبًا باللهجة اللبنانية بأحرف إنجليزية، ترجم السؤال إلى العربية الفصحى بأكثر طريقة احترافية ممكنة مع الحفاظ على المهنة في التعبير."
        "ابدأ داخليًا بخطة مختصرة من ٣ إلى ٥ خطوات مفاهيمية لمعالجة كل مرحلة من مراحل السؤال، ولا تضمن هذه الخطة في النتيجة النهائية. "
        "أعد كتابة السؤال بنفس الصيغة المستخدمة من قبل المتكلم (لا تغير الضمائر أو وجهة النظر)، ولا تضف أو تحذف أي معنى جديد."
        "إذا كان السؤال واضحًا ومباشرًا، أعِد عرضه كما هو مع تحسين طفيف للأسلوب فقط."
        "الهدف هو جعل السؤال أوضح وأكثر رسمية دون تغيير معناه أو صيغة المتكلم. "
        "بعد تعديل كل سؤال، تحقق داخليًا في جملة أو جملتين أن التعديل حقق الوضوح والاحترافية دون تغيير الجوهر. "
        "اكتب فقط الصيغة النهائية للسؤال دون شرح أو خطوات."
        "الإخراج دائمًا عبارة عن السؤال النهائي المعاد صياغته فقط (جملة واحدة أو أكثر باللغة العربية الفصحى). لا تشرح أو تدرج أي تفاصيل عن العملية أو القوائم المنفذة داخليًا — الناتج النهائي هو السؤال فقط."
        )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1,
    )
    return response.choices[0].message.content.strip()

In [22]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_answer_with_history(user_id, query, retrieved_chunks, formatted_history: str):
    model="gpt-4o-mini"
    context = "\n\n".join([c["chunk"] for c in retrieved_chunks])
    query = reformulate_query(query)
    messages = [
        {
            "role": "system",
            "content": (
                "أنت مساعد يجيب على أنه السيد هاشم صفي الدين."
                "استعن داخليًا بقائمة مختصرة (3-7 عناصر) للخطوات المفاهيمية قبل تقديم أي إجابة، لكن لا تطبع أو تدرج هذه القائمة في الرد."
                "عند التعامل مع أسئلة تحية أو أسئلة عامة بسيطة مثل السلام عليكم أو كيف الحال، يمكن الإجابة عليها مباشرة دون الحاجة للاعتماد على نصوص السياق."
                "اعتمد في إجابتك فقط على النصوص المتوفرة في السياق للأسئلة الأخرى. إذا لم يكن الجواب واضحًا وكاملاً في السياق، قل إنه لا يوجد إجابة."
                ".لا يجب أن تجيب على أي سؤال إذا لم يتم استرجاع أية مقاطع نصية، ولا تذكر أي موضوع في إجاباتك ما لم يكن موجودًا أيضًا في المقاطع المسترجعة."
                "لا تشر إلى السياق في إجابتك إذ أن القارئ لا يستطيع قراءته."
                "تحدث باحترام عن الشخصيات الشيعية، مع ذكر الألقاب المناسبة."
                "أجب دائمًا باللغة العربية."
                "لا تشرح أو تدرج أي تفاصيل عن العملية أو القوائم المنفذة داخليًا، اكتب الجواب فقط."
                "قبل عرض الاجابة للقارئ تاكد داخليًا من ان الاجابة دقيقة ومكتملة بناء على المعلومات المتوفرة في السياق، وملتزمة بكافة الشروط والتعليمات الواردة."
            ),
        },
        {"role": "system", "content": PERSONA_PREAMBLE},
        {"role": "system", "content": f"\n\nالرسائل السابقة:\n{formatted_history}"},
        {
            "role": "user",
            "content": f"السياق:\n\n{context}\n\nالسؤال: {query}",
        },    
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1,
    )
    print(messages)
    answer_text = response.choices[0].message.content.strip()

    # Build unique citations list from retrieved chunks' metadata
    citations = []

    if retrieved_chunks:
        seen = set()
        for item in retrieved_chunks:
            md = item.get("metadata", {})
            title = (md.get("title") or "").strip()
            source = (md.get("source") or "").strip()
            key = (title, source)
            if (title or source) and key not in seen:
                seen.add(key)
                if title and source:
                    citations.append(f"- {title} — {source}")
                elif title:
                    citations.append(f"- {title}")
                else:
                    citations.append(f"- {source}")

    if citations:
        answer_text = f"{answer_text}\n\nالمصادر:\n" + "\n".join(citations)

    print(answer_text)
    return answer_text

In [10]:
with open("/Users/mohamad/Documents/GitHub/Personalized-RAG-Chatbot/character.json", "r", encoding="utf-8") as f:
    character = json.load(f)

def build_persona_preamble(c) -> str:
    if isinstance(c, list):
        c = next((x for x in c if isinstance(x, dict) and 'lexicon' in x), (c[0] if c and isinstance(c[0], dict) else {}))
    elif not isinstance(c, dict):
        c = {}

    role_instructions = c.get("role_instructions")
    t = c.get("tone", {})
    tone = ", ".join(t.values())

    lex = c.get("lexicon") or {}
    inv = "\n- ".join(lex.get("invocations") or [])
    honors = "\n- ".join(lex.get("honorifics") or [])
    ashura = "\n- ".join(lex.get("ashura_register") or [])
    bins = "\n- ".join(lex.get("binaries") or [])
    values = "\n- ".join(lex.get("values") or [])

    dm_formal = "\n- ".join(lex.get("discourse_markers_formal") or [])
    dm_colloq = "\n- ".join(lex.get("discourse_markers_colloquial") or [])
    emph = "\n- ".join(lex.get("emphasis_markers") or [])
    key_terms = "\n- ".join(lex.get("key_terms") or [])

    reh = c.get("rhetorical_scaffold") or {}
    open = "\n- ".join(reh.get("open") or [])
    develop = "\n- ".join(reh.get("develop") or [])
    evidence = "\n- ".join(reh.get("evidence") or [])
    application = "\n- ".join(reh.get("application") or [])
    closure = "\n- ".join(reh.get("closure") or [])

    pacing = c.get("response_pacing", {})
    response_pacing = ", ".join(pacing.values())

    greetings = "\n- ".join(c.get("greeting_templates") or [])
    closing = "\n- ".join(c.get("closing_templates") or [])
    condolences = "\n- ".join(c.get("condolence_templates") or [])

    q = c.get("quote_frames", {})
    quote_frames = ", ".join(q.values())

    do = "\n- ".join(c.get("do") or [])
    dont = "\n- ".join(c.get("dont") or [])

    snippet = c.get("style_snippets", {})
    style_snippets = ", ".join(snippet.values())

    micro = c.get("micro_templates", {})
    micro_templates = ", ".join(micro.values())

    topics = "\n- ".join(c.get("topics") or [])

    tk = c.get("topics_knowledge") or {}
    personal_section = ""
    other_topics_sections = []
    if isinstance(tk, dict):
        for name, data in tk.items():
            is_personal = isinstance(name, str) and "سيرة السيد هاشم صفيّ الدين" in name
            highlights = []
            points = []
            use_with = None
            if isinstance(data, dict):
                highlights = data.get("highlights") or []
                points = data.get("points") or []
                use_with = data.get("use_with")
            if is_personal:
                lines = []
                if points:
                    lines.extend(points)
                elif highlights:
                    lines.extend(highlights)
                else:
                    lines.extend([f"{k}: {v}" for k, v in data.items()])
                personal_section = "\n".join(["السيرة الشخصية:"] + [f"- {x}" for x in lines])
            else:
                lines = []
                if highlights:
                    lines.extend(highlights)
                elif points:
                    lines.extend(points)
                else:
                    lines.extend([f"{k}: {v}" for k, v in data.items()])
                section = "\n".join([name + ":"] + [f"- {x}" for x in lines] + ([f"- use_with: {use_with}"] if use_with else []))
                other_topics_sections.append(section)
    topics_knowledge_personal = personal_section
    topics_knowledge_other = "\n\n".join(other_topics_sections)

    cu = c.get("contextual_usage") or {}
    contextual_usage = "\n".join(["قيود الاستخدام السياقي (إلزامي):"] + [f"- {k}: {v}" for k, v in cu.items()])

    return (
        f"الشخصية: {c.get('name','')}\n"
        f"الغرض: {c.get('purpose','')}\n"
        f"تعليمات الدور: {role_instructions}\n"
        f"النبرة: {tone}\n"
        f"الافتتاحيات:\n- {inv}\n"
        f"الألقاب:\n- {honors}\n"
        f"سجل عاشورائي:\n- {ashura}\n"
        f"الثنائيات:\n- {bins}\n"
        f"القيم:\n- {values}\n"
        f"روابط الخطاب (فصحى):\n- {dm_formal}\n"
        f"روابط الخطاب (عامية):\n- {dm_colloq}\n"
        f"علامات التأكيد:\n- {emph}\n"
        f"مصطلحات مفتاحية:\n- {key_terms}\n"
        f"التمهيد البلاغي:\n- {open}\n"
        f"التطوير البلاغي:\n- {develop}\n"
        f"أمثلة وأدلة:\n- {evidence}\n"
        f"تطبيق البلاغة:\n- {application}\n"
        f"الإغلاق البلاغي:\n- {closure}\n"
        f"إيقاع الاستجابة: {response_pacing}\n"
        f"قوالب الترحيب:\n- {greetings}\n"
        f"قوالب الختام:\n- {closing}\n"
        f"قوالب التعزية:\n- {condolences}\n"
        f"أطر الاقتباس: {quote_frames}\n"
        f"افعل:\n- {do}\n"
        f"لا تفعل:\n- {dont}\n"
        f"مقتطفات أسلوبية: {style_snippets}\n"
        f"قوالب دقيقة: {micro_templates}\n"
        f"الموضوعات:\n- {topics}\n"
        f"{topics_knowledge_personal}\n\n{topics_knowledge_other}\n"
        f"{contextual_usage}\n"
        f"تنبيه: الالتزام بما سبق إلزامي في كل إجابة."
    )

PERSONA_PREAMBLE = build_persona_preamble(character)

In [23]:
query = "ما هي العلامات التي تدل على استجابة دعاء الإنسان؟"
query = "كيف يمكن للإنسان أن يعرف أن دعاءه قد استجيب؟"
query = "ما هو رأيك فيما حدث بالأمس؟"

results = search_index(query)

retrieved_chunks = results
answer = generate_answer_with_history(user_id=1, query=query, retrieved_chunks=retrieved_chunks, formatted_history="")
print(answer)

[{'role': 'system', 'content': 'أنت مساعد يجيب على أنه السيد هاشم صفي الدين.استعن داخليًا بقائمة مختصرة (3-7 عناصر) للخطوات المفاهيمية قبل تقديم أي إجابة، لكن لا تطبع أو تدرج هذه القائمة في الرد.عند التعامل مع أسئلة تحية أو أسئلة عامة بسيطة مثل السلام عليكم أو كيف الحال، يمكن الإجابة عليها مباشرة دون الحاجة للاعتماد على نصوص السياق.اعتمد في إجابتك فقط على النصوص المتوفرة في السياق للأسئلة الأخرى. إذا لم يكن الجواب واضحًا وكاملاً في السياق، قل إنه لا يوجد إجابة..لا يجب أن تجيب على أي سؤال إذا لم يتم استرجاع أية مقاطع نصية، ولا تذكر أي موضوع في إجاباتك ما لم يكن موجودًا أيضًا في المقاطع المسترجعة.لا تشر إلى السياق في إجابتك إذ أن القارئ لا يستطيع قراءته.تحدث باحترام عن الشخصيات الشيعية، مع ذكر الألقاب المناسبة.أجب دائمًا باللغة العربية.لا تشرح أو تدرج أي تفاصيل عن العملية أو القوائم المنفذة داخليًا، اكتب الجواب فقط.قبل عرض الاجابة للقارئ تاكد داخليًا من ان الاجابة دقيقة ومكتملة بناء على المعلومات المتوفرة في السياق، وملتزمة بكافة الشروط والتعليمات الواردة.'}, {'role': 'system', 'content'

In [24]:
history = "{'role': 'system', 'content': '\n\nالرسائل السابقة:\nuser: aan shu btaaref tehke'}, {'role': 'user', 'content': 'السياق:\n\n\n\nالسؤال: عن ماذا تستطيع التحدث؟'}]"
query = "hello"
# query =  "سلام"

refined_query = reformulate_query(query)
print("🔄 Reformulated:", refined_query)


🔄 Reformulated: مرحبًا، كيف يمكنني مساعدتك اليوم؟


In [1]:
import re
import os
from openai import AsyncOpenAI

# Arabic harakat characters
HARAKAT = "".join([
    "\u064b", # tanween fath
    "\u064c", # tanween damm
    "\u064d", # tanween kasr
    "\u064e", # fatha
    "\u064f", # damma
    "\u0650", # kasra
    "\u0651", # shadda
    "\u0652", # sukun
])

client = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

messages = [
    {"role": "system", "content": """
        You are a professional Arabic linguist. 
        Your task is to take Arabic input text and add only the necessary short vowels (حركات) to improve pronunciation for text-to-speech audio generation. 

        Rules:
        1. Add vowels only when needed to clarify correct pronunciation. 
        2. Do NOT add full grammatical case endings unless they are essential to avoid ambiguity. 
        3. Absolutely do NOT add a vowel mark on the final letter of the last word in any sentence, phrase, or clause. Leave that letter bare. 
        4. Keep proper nouns, Qur'anic verses, and classical Arabic phrases fully vocalized, but still omit the final vowel on the last letter of the sentence. 
        5. Return only the diacritized Arabic text without explanations or translations.
    """},
    {"role": "user", "content": "ألف لام ميم. كاف هاء ياء عين صاد"}
]

response = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0.3,
)

output = response.choices[0].message.content.strip()

# 🔥 Post-process: remove any harakat from final letter of each sentence
def strip_final_harakat(text: str) -> str:
    # split by sentence-ending punctuation (., ؟, !, …)
    sentences = re.split(r'([\.؟!\u061F])', text)
    cleaned = []
    for i in range(0, len(sentences), 2):
        sentence = sentences[i]
        punct = sentences[i+1] if i+1 < len(sentences) else ""
        if sentence.strip():
            # remove harakat from last non-space char
            sentence = re.sub(f"([{HARAKAT}]+)$", "", sentence)
        cleaned.append(sentence + punct)
    return "".join(cleaned)

final_output = strip_final_harakat(output)
print(final_output)


أَلِف لَام مِيم. كَاف هَاء يَاء عَيْن صَاد


In [ ]:
muqattaat_dict = {
    "ا": "أَلِفْ",
    "ل": "لَامْ",
    "م": "مِيمْ",
    "ص": "صَادْ",
    "ر": "رَا",
    "ك": "كَافْ",
    "ه": "هَا",
    "ي": "يَا",
    "ع": "عَيْنْ",
    "ط": "طَا",
    "س": "سِينْ",
    "ح": "حَا",
    "ق": "قَافْ",
    "ن": "نُونْ"
}


def process_muqattaat(text):
    """
    Detects sequences like الم, ن, كهيعص and converts them into their recitation forms.
    """
    result = []
    for char in text:
        if char in muqattaat_dict:
            result.append(muqattaat_dict[char])
        else:
            result.append(char)  # Keep normal text as-is
    return " ".join(result)

# Example usage
examples = ["الم", "ن", "كهيعص", "طه", "المص", "الر", "المر", "طسم", "طس", "يس", "ص", "حم", "عسق", "ق"]

for ex in examples:
    print(f"{process_muqattaat(ex)}.")


أَلِفْ لَامْ مِيمْ.
نُونْ.
كَافْ هَا يَا عَيْنْ صَادْ.
طَا هَا.
أَلِفْ لَامْ مِيمْ صَادْ.
أَلِفْ لَامْ رَا.
أَلِفْ لَامْ مِيمْ رَا.
طَا سِينْ مِيمْ.
طَا سِينْ.
يَا سِينْ.
صَادْ.
حَا مِيمْ.
عَيْنْ سِينْ قَافْ.
قَافْ.


In [ ]:
# example.py
import os
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs

load_dotenv()

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

file_path = "..."

with open(file_path, "rb") as audio_file:
    transcription = elevenlabs.speech_to_text.convert(
        file=audio_file,
        model_id="scribe_v1",
        tag_audio_events=True,
        language_code="ar",
        diarize=True,
    )

print(transcription)

language_code='ara' language_probability=1.0 text='أعوذ بالله من الشيطان الرجيم. بسم الله الرحمن الرحيم. والصلاة والسلام على سيدنا أبي القاسم محمد وعلى آل بيته الطيبين الطاهرين. ترددتنا عن كلمة بسم الله الرحمن الرحيم التي ترد بمقدمة كل سورة في القرءان الكريم. وبدأنا بالأية الأولى من سورة الحجرات، حيث ذكرنا أن هذه السور إجمالا تتعرض لبعض القواعد والأحكام والآداب السلوكية والأخلاقية والتربوية لبناء الفرد والمجتمع، وهي هذه السورة على صغر حجمها 18 اية، لكنها مليئة بالمفاهيم والقيم التأسيسية للمجتمع الإسلامي. وفي ذلك الوقت في أيام رسول الله صلى الله عليه وسلم كان هذا المجتمع مجتمعا حديث العهد بالإسلام والتدين والمعرفة بالأديان السماوية بشكل عام، فاقتضى التفصيل والتبعين والتحديد. لذا نجد أن هذه السور في بعض الأحيان تتعرض لقيم عامة وكبيرة ومهمة، وفي بعض الأحيان تتعرض لتفاصيل من أجل تحديد ما هو المطلوب بشكل مباشر، وهذا ما يحتاجه الفرد المسلم والمجتمع المسلم. في الأية الأولى. يا أيها الذين امنوا لا تقدموا بين يدي الله ورسوله واتقوا الله إن الله سميع عليم. قلنا أن المفسرين ذهبوا في قوله تعالى لا

In [17]:
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
import os
load_dotenv()

client = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

audio = client.text_to_speech.convert(
    text="السلام عليكم.",
    voice_id="FeEvUACvySn9x0TMXIxQ",
    # FeEvUACvySn9x0TMXIxQ
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128",
)

audio_bytes = b"".join(audio)

with open("output.mp3", "wb") as f:
    f.write(audio_bytes)

print("✅ Audio saved as output.mp3")

ApiError: headers: {'date': 'Wed, 24 Sep 2025 07:49:44 GMT', 'server': 'uvicorn', 'content-length': '113', 'content-type': 'application/json', 'access-control-allow-origin': '*', 'access-control-allow-headers': '*', 'access-control-allow-methods': 'POST, PATCH, OPTIONS, DELETE, GET, PUT', 'access-control-max-age': '600', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-trace-id': '2473bfed410f229ea2f9b03423a61b78', 'x-region': 'us-central1', 'via': '1.1 google, 1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 404, body: {'detail': {'status': 'voice_not_found', 'message': 'A voice with the voice_id FeEvUACvySn9x0TMXIxQ was not found.'}}

09196800213